In [1]:
%reset -f
%whos

Interactive namespace is empty.


add nn.DataParallel and double check that the data obtained in dataloader for concat and sep are actually what you think they are

In [3]:
import os
import esm
from os.path import exists
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.functional import log_softmax, pad
from torch.autograd import Variable
import matplotlib.pyplot as plt
import math
import copy
import time
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import re
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
from torch.utils.data import Dataset
import warnings
import seaborn
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
#from transformers import EsmTokenizer, EsmModel
from torch.utils.data import dataset
from sklearn.metrics import roc_auc_score, average_precision_score

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
#peptide_10mer = pd.read_csv("MS10_dat_dummy_seed_757_5cv_x10p_train_test_valid.txt", sep = " ")
peptide_10mer_subset = pd.read_csv("MS10_dat_dummy_seed_757_5cv_x10p_train_test_valid.txt", sep = " ")

In [8]:
'''samples_per_fold_and_set = []

# Iterate through each fold and set type to sample 5 rows each
for fold in range(5):  # Folds 0 through 4
    for set_type in ['train', 'valid']:  # Assuming you meant 'val' instead of 'valid' as per your requirement
        # Sample 5 rows where the fold and set type match the current iteration
        samples = peptide_10mer[(peptide_10mer['fold'] == fold) & (peptide_10mer['set'] == set_type)].sample(n=5)
        samples_per_fold_and_set.append(samples)

# Concatenate all the sampled DataFrames
peptide_10mer_subset = pd.concat(samples_per_fold_and_set, ignore_index=True)

# Optional: Filter the columns if necessary (seems you want to keep all relevant columns)
peptide_10mer_subset = peptide_10mer_subset[["allele", "fold", "set", "seq", "label"]]

# Display the counts to verify the distribution
print(peptide_10mer_subset[["fold", "set"]].value_counts())'''

'samples_per_fold_and_set = []\n\n# Iterate through each fold and set type to sample 5 rows each\nfor fold in range(5):  # Folds 0 through 4\n    for set_type in [\'train\', \'valid\']:  # Assuming you meant \'val\' instead of \'valid\' as per your requirement\n        # Sample 5 rows where the fold and set type match the current iteration\n        samples = peptide_10mer[(peptide_10mer[\'fold\'] == fold) & (peptide_10mer[\'set\'] == set_type)].sample(n=5)\n        samples_per_fold_and_set.append(samples)\n\n# Concatenate all the sampled DataFrames\npeptide_10mer_subset = pd.concat(samples_per_fold_and_set, ignore_index=True)\n\n# Optional: Filter the columns if necessary (seems you want to keep all relevant columns)\npeptide_10mer_subset = peptide_10mer_subset[["allele", "fold", "set", "seq", "label"]]\n\n# Display the counts to verify the distribution\nprint(peptide_10mer_subset[["fold", "set"]].value_counts())'

In [9]:
one_hot = pd.read_csv("aafeatmat_onehot_ext.txt", sep = " ")
one_hot = one_hot.rename(index={'.': 'B'}, columns = {'.': 'B'})
one_hot_dict = {index: row.tolist() for index, row in one_hot.iterrows()}

kf = pd.read_csv("aafeatmat_KideraFactors.txt", sep = " ")

pc = pd.read_csv("aafeatmat_AAPropsPCA3.txt", sep = " ")
pc.loc["B"] = 0
pc_dict = {index: row.tolist() for index, row in pc.iterrows()}

hla_full_pc_kf_pseudoseq_wseq = pd.read_csv("ABCG_prot.parsed.clean.updated.ALL.FEATS.txt", sep = " ", index_col = 1)

hla_seq_df = hla_full_pc_kf_pseudoseq_wseq.seq.copy()
hla_seq_df_dict = {index: row.tolist() for index, row in pd.DataFrame(hla_seq_df).iterrows()}

amino_acid_mapping = { #B = BOS token
    'B': 0, 'A': 1, 'R': 2, 'N': 3, 'D': 4, 'C': 5,
    'Q': 6, 'E': 7, 'G': 8, 'H': 9, 'I': 10,
    'L': 11, 'K': 12, 'M': 13, 'F': 14, 'P': 15,
    'S': 16, 'T': 17, 'W': 18, 'Y': 19, 'V': 20, "-":21
}

In [10]:
def add_padding(sequence, max_len, padding_token = "-"):
    """
    Pads the amino acid sequence to a maximum length by adding the necessary number of padding tokens ("-") after the fourth amino acid

    Parameters: 
    - sequence (str): input amino acid sequence for peptide
    - max_len (int): maximum length you want to pad the sequence to 
    - padding_token (str): token used for padding
    
    Returns: 
    - str: padded amino acid sequence
    """
    difference = max_len - len(sequence)
    final_seq = sequence[:4] + padding_token*difference + sequence[4:]
    return(final_seq)
    
def get_param(df, tgt_col, pep_col, hla_col):
    y_data = df[tgt_col]
    x_data = df[pep_col]
    hla_name = df[hla_col]
    max_len = max_len = max([len(x) for x in x_data])
    return y_data, x_data, hla_name, max_len

def initialize_param(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

class PositionalEncoding(nn.Module): #from https://pytorch.org/tutorials/beginner/transformer_tutorial.html#:~:text=A%20sequence%20of%20tokens%20are,TransformerEncoderLayer.

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1) #according to chatgpt, did not need to add "device = device" here since i have register_buffer 
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [20]:
##0322: addded  esm_model, esm_batch_converter, esm_alphabet, here

def transform_peptide_hla_sep(hla_seq, peptide_seq, max_len, esm_model, esm_batch_converter, esm_alphabet, encode_choice="embed"):
    padded_seq = add_padding(peptide_seq, max_len, "-")
    
    #converted this code to be for just one peptide since that is what the dataloader needs
    if encode_choice == "embed":
        peptide_processed = torch.tensor([amino_acid_mapping[aa] for aa in padded_seq])
        hla_processed = torch.tensor([amino_acid_mapping[aa] for aa in hla_seq])
        padding_mask = (peptide_processed != amino_acid_mapping["-"]).unsqueeze(1)
        
    elif encode_choice == "esm":
        padded_seq = add_padding(peptide_seq, max_len, "<pad>")
        peptide_input = [("Peptide_Input", padded_seq)]
        hla_input = [("HLA_Input", hla_seq)]
        
        peptide_batch_labels, peptide_batch_strs, peptide_batch_tokens = esm_batch_converter(peptide_input)
        hla_batch_labels, hla_batch_strs, hla_batch_tokens = esm_batch_converter(hla_input)

        # Extract per-residue representations (on CPU)
        with torch.no_grad():
            peptide_results = esm_model(peptide_batch_tokens, repr_layers=[33], return_contacts=True)
            hla_results = esm_model(hla_batch_tokens, repr_layers=[33], return_contacts=True)
        
        peptide_processed = peptide_results["representations"][33]
        hla_processed = hla_results["representations"][33]
        
        #this is the padding mask for the transformer (padding for the esm model has already been incorporated into the <pad> token for esm_model to deal with - double check! TODO)
        padding_mask = (peptide_batch_tokens == esm_alphabet.padding_idx).unsqueeze(1) #QUESTION/TODO: Do I need to unsqueeze this? this is currently [batch_size, seq_length]
        
    elif encode_choice == "one_hot":
        peptide_processed = torch.tensor([one_hot_dict[aa] for aa in padded_seq])
        hla_processed = torch.tensor([one_hot_dict[aa] for aa in hla_seq])
        padding_bool = [aa != "-" for aa in padded_seq]
        padding_mask = torch.tensor(padding_bool).unsqueeze(1)

    elif encode_choice == "pc":
        peptide_processed = torch.tensor([pc_dict[aa] for aa in padded_seq])
        hla_processed = torch.tensor([pc_dict[aa] for aa in hla_seq])
        padding_bool = [aa != "-" for aa in padded_seq]
        padding_mask = torch.tensor(padding_bool).unsqueeze(1)
    else:
        raise ValueError(f"Unsupported encoding choice: {encode_choice}. Must be 'one_hot', 'esm', embed', or 'pc'")
    
    return hla_processed, peptide_processed, padding_mask

#-----------------------------------------------------------
#before using esm, need to define esm_model and esm_batch_converter: 
#esm_model, esm_alphabet = esm.pretrained.esm2_t33_650M_UR50D()
#esm_batch_converter = esm_alphabet.get_batch_converter()
#TODO: according to https://github.com/facebookresearch/esm/blob/main/esm/constants.py#L7, "-" is recognized by the model as <unk>, not <pad> so I might make that more explicit somehow --> TODO
#NOTE: according to https://github.com/search?q=repo%3Afacebookresearch%2Fesm%20%3Cpad%3E&type=code, "<pad>" is used to get the padding_idx (and in the above, you see that it corresponds to 1 and the padded parts also correspond to 1 at the end of the seq)
#NOTE: changed this such that aa == "-" is used for padding mask instead of aa != "-" since usually True means ignore the position and False means don't ignroe the position
def transform_peptide_hla_concat(hla_seq, peptide_seq, max_len, esm_model, esm_batch_converter, esm_alphabet, encode_choice="embed"):
    padded_seq = add_padding(peptide_seq, max_len, "-")
    concat_seq = padded_seq + hla_seq
    
    #TODO: for all of these, check that it should be unsqueeze(1)
    #converted this code to be for just one peptide since that is what the dataloader needs
    if encode_choice == "embed":
        concat_processed = torch.tensor([amino_acid_mapping[aa] for aa in concat_seq])
        padding_mask = (concat_processed == amino_acid_mapping["-"]).unsqueeze(1)
        
    #TODO: check if this is correctly implemented for a one peptide at a time basis 
    elif encode_choice == "esm":
        padded_seq = add_padding(peptide_seq, max_len, "<pad>")
        concat_seq = padded_seq + hla_seq
        concat_input = [("Concat_Input", concat_seq)]
        concat_batch_labels, concat_batch_strs, concat_batch_tokens = esm_batch_converter(concat_input)
        #concat_batch_lens = (concat_batch_tokens != esm_alphabet.padding_idx).sum(1) #only need this if you want to later use the means instead of the token representations for each amino acid
        
        # Extract per-residue representations (on CPU)
        with torch.no_grad():
            concat_results = esm_model(concat_batch_tokens, repr_layers=[33], return_contacts=True)
        concat_processed = concat_results["representations"][33]
        #this is the padding mask for the transformer (padding for the esm model has already been incorporated into the <pad> token for esm_model to deal with - double check! TODO)
        padding_mask = (concat_batch_tokens == esm_alphabet.padding_idx).unsqueeze(1) #QUESTION/TODO: Do I need to unsqueeze this? this is currently [batch_size, seq_length]

    elif encode_choice == "one_hot":
        concat_processed = torch.tensor([one_hot_dict[aa] for aa in concat_seq])
        padding_bool = [aa == "-" for aa in concat_seq]
        padding_mask = torch.tensor(padding_bool).unsqueeze(1)

    elif encode_choice == "pc":
        concat_processed = torch.tensor([pc_dict[aa] for aa in concat_seq])
        padding_bool = [aa == "-" for aa in concat_seq]
        padding_mask = torch.tensor(padding_bool).unsqueeze(1)
    else:
        raise ValueError(f"Unsupported encoding choice: {encode_choice}. Must be 'one_hot', 'esm', embed', or 'pc'")
    
    return concat_processed, padding_mask


In [12]:
#use this if you want to create the train dataset automatically and can input the whole original file (no need to separate out the train and validation from each other beforehand)
class PeptideHLADataset_Training_Validation(Dataset):
    def __init__(self, input_df, hla_seq_df, tgt_col, pep_col, hla_col, fold_num_val, train_or_val, max_len, concat_or_sep, encode_choice = "embed", transform=None, esm_model=None, esm_batch_converter=None, esm_alphabet=None):
        self.input_df = input_df
        self.hla_seq_df = hla_seq_df
        self.tgt_col = tgt_col
        self.pep_col = pep_col
        self.hla_col = hla_col
        self.max_len = max_len
        self.fold_num_val = fold_num_val
        self.train_or_val = train_or_val
        self.encode_choice = encode_choice
        self.transform = transform
        self.train_df = input_df[(input_df['fold'] == fold_num_val) & (input_df['set'] == "train")]
        self.val_df = input_df[(input_df['fold'] == fold_num_val) & (input_df['set'] == "valid")]
        #self.train_df = input_df[input_df['fold'] != fold_num_val]
        #self.val_df = input_df[input_df['fold'] == fold_num_val]
        self.esm_model = esm_model
        self.esm_batch_converter = esm_batch_converter
        self.esm_alphabet = esm_alphabet
        self.concat_or_sep = concat_or_sep

    def __len__(self):
        if self.train_or_val == "train":
            return len(self.train_df)
        elif self.train_or_val == "validation":
            return len(self.val_df)

    def __getitem__(self, idx):
        print(idx)
        if torch.is_tensor(idx):
            idx = idx.tolist()
        #print(idx)
        if self.train_or_val == "train":
            sample = self.train_df.iloc[idx]
        elif self.train_or_val == "validation":
            sample = self.val_df.iloc[idx]
        #print(sample.shape)   
        peptide = sample[self.pep_col]
        hla_name = sample[self.hla_col]
        label = sample[self.tgt_col]
        hla_sequence = self.hla_seq_df[hla_name]

        if self.transform:
            if self.concat_or_sep == "concat":
                concat_processed, padding_mask = self.transform(hla_sequence, peptide, self.max_len, self.esm_model, self.esm_batch_converter, self.esm_alphabet, self.encode_choice)
                #TODO/CHECK: added this to get rid of the extra dim at the front 
                #return concat_processed, padding_mask, label #original - use w a custom collate fn
                #print("dataset")
                #print(concat_processed.squeeze(0).shape)
                #print(padding_mask.shape)
                return concat_processed.squeeze(0), padding_mask, label
            
            elif self.concat_or_sep == "sep":
                hla_sequence, peptide, padding_mask = self.transform(hla_sequence, peptide, self.max_len, self.esm_model, self.esm_batch_converter, self.esm_alphabet, self.encode_choice)
                #return hla_sequence, peptide, padding_mask, label #original - use w a custom collate fn
                return hla_sequence.squeeze(0), peptide.squeeze(0), padding_mask, label
            
            else:
                raise ValueError("'concat_or_sep' parameter must be 'concat' or 'sep'.")
        else: 
            raise RuntimeError("Transform function is not defined. Please define it.")

def get_data_loaders(input_df, hla_seq_df, tgt_col, pep_col, hla_col, fold_num_val, max_len, esm_model=None, esm_batch_converter=None, esm_alphabet=None, batch_size = 32, encode_choice = 'embed', concat_or_sep = "concat", collate_fn=None):
    if concat_or_sep == "concat":
        transform_function = transform_peptide_hla_concat
    elif concat_or_sep == "sep":
        transform_function = transform_peptide_hla_sep
    else:
        raise ValueError("concat_or_sep argument must be either 'concat' or 'sep'")
        
    train_dataset = PeptideHLADataset_Training_Validation(input_df, hla_seq_df, tgt_col, pep_col, hla_col, fold_num_val, "train", max_len = max_len, transform = transform_function, concat_or_sep = concat_or_sep, encode_choice = encode_choice, esm_model=esm_model, esm_batch_converter=esm_batch_converter, esm_alphabet=esm_alphabet)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = PeptideHLADataset_Training_Validation(input_df, hla_seq_df, tgt_col, pep_col, hla_col, fold_num_val, "validation", max_len = max_len, transform = transform_function, concat_or_sep = concat_or_sep, encode_choice = encode_choice, esm_model=esm_model, esm_batch_converter=esm_batch_converter, esm_alphabet=esm_alphabet)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

In [15]:
def train_model(model, train_loader, model_type, optimizer, criterion, epoch, concat_or_sep, pooling_strat):
    model.train()
    #print(1)
    #print(pooling_strat)
    running_loss = 0.0
    if concat_or_sep == "concat":
        #print("training start")
        for batch_idx, (concat_processed, padding_mask, labels) in enumerate(train_loader):
            #added this for gpu usage: 
            concat_processed, padding_mask, labels = concat_processed.to(device), padding_mask.to(device), labels.to(device)
            optimizer.zero_grad()
            #outputs = model(concat_processed.float(), model_type = model_type, src_key_padding_mask = padding_mask).squeeze() #(reduce a dim so it matches the dim of the labels [32] instead of [32, 1])            
            #outputs = model(concat_processed.float(), pooling_strat = pooling_strat, model_type = model_type, src_key_padding_mask = padding_mask).squeeze() #(reduce a dim so it matches the dim of the labels [32] instead of [32, 1])        
            #outputs = model(concat_processed.float(), pooling_strat, padding_mask, model_type).squeeze() #(reduce a dim so it matches the dim of the labels [32] instead of [32, 1])            
            outputs = model(concat_processed.float(), pooling_strat = pooling_strat, model_type = model_type, src_key_padding_mask = padding_mask.squeeze(1).squeeze(1)).squeeze() #(reduce a dim so it matches the dim of the labels [32] instead of [32, 1])        
            loss = criterion(outputs, labels.float())

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if batch_idx % 100 == 99:  #Print average loss every 10 mini-batches
                print(f'Training Epoch: {epoch + 1}, Batch: {batch_idx + 1}, Loss: {running_loss / 100:.4f}')
                running_loss = 0.0
        #print("training done")
    elif concat_or_sep == "sep":
        #print(2)
        #print(pooling_strat)
        for batch_idx, (hla_processed, peptide_processed, padding_mask, labels) in enumerate(train_loader):
            #added this for gpu usage: 
            hla_processed, peptide_processed, padding_mask, labels = hla_processed.to(device), peptide_processed.to(device), padding_mask.to(device), labels.to(device)
            optimizer.zero_grad()
            #outputs = model(hla_processed.long(), peptide_processed.long(), pooling_strat = pooling_strat, model_type = model_type, src_key_padding_mask = padding_mask).squeeze() #(reduce a dim so it matches the dim of the labels [32] instead of [32, 1])
            #outputs = model(hla_processed.long(), peptide_processed.long(), pooling_strat, padding_mask, model_type).squeeze()
            outputs = model(hla_processed.float(), peptide_processed.float(), pooling_strat = pooling_strat, model_type = model_type, src_key_padding_mask = padding_mask.squeeze(1).squeeze(1)).squeeze() #(reduce a dim so it matches the dim of the labels [32] instead of [32, 1])
            loss = criterion(outputs, labels.float())

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if batch_idx % 100 == 99:  #Print average loss every 10 mini-batches
                print(f'Training Epoch: {epoch + 1}, Batch: {batch_idx + 1}, Loss: {running_loss / 100:.4f}')
                running_loss = 0.0
    else:
        raise ValueError("concat_or_sep argument must be either 'concat' or 'sep'")
    
'''def eval_model(model, val_loader, model_type, threshold, criterion, concat_or_sep, pooling_strat):
    model.eval()
    total_correct, total_samples = 0, 0
    
    if concat_or_sep == "concat":
        with torch.no_grad():
            for batch_idx, (concat_processed, padding_mask, labels) in enumerate(val_loader):
                #IMPT: only have the squeeze here if you are doing binary classification w sigmoid classification! if you use softmax for 2 classes, take this out
                #outputs = model(concat_processed.long(), pooling_strat = pooling_strat, model_type = model_type, src_key_padding_mask = padding_mask).squeeze()
                #outputs = model(concat_processed.long(), pooling_strat, padding_mask, model_type).squeeze()
                outputs = model(concat_processed.float(), pooling_strat = pooling_strat, model_type = model_type, src_key_padding_mask = padding_mask.squeeze(1).squeeze(1)).squeeze() #(reduce a dim so it matches the dim of the labels [32] instead of [32, 1])        
                #predicted = torch.argmax(output, dim=1) #for softmax
                predicted = (outputs >= threshold).long() #for sigmoid
                total_correct += (predicted == labels).sum().item()
                total_samples += labels.size(0)
    elif concat_or_sep == "sep":
        with torch.no_grad():
            for batch_idx, (hla_processed, peptide_processed, padding_mask, labels) in enumerate(val_loader):
                #IMPT: only have the squeeze here if you are doing binary classification w sigmoid classification! if you use softmax for 2 classes, take this out
                #outputs = model(hla_processed.long(), peptide_processed.long(), pooling_strat = pooling_strat, model_type = model_type, src_key_padding_mask = padding_mask).squeeze()
                #outputs = model(hla_processed.long(), peptide_processed.long(), pooling_strat, padding_mask, model_type).squeeze()
                outputs = model(hla_processed.float(), peptide_processed.float(), pooling_strat = pooling_strat, model_type = model_type, src_key_padding_mask = padding_mask.squeeze(1).squeeze(1)).squeeze()
                
                #predicted = torch.argmax(output, dim=1) #for softmax
                predicted = (outputs >= threshold).long() #for sigmoid
                total_correct += (predicted == labels).sum().item()
                total_samples += labels.size(0)
    else:
        raise ValueError("concat_or_sep argument must be either 'concat' or 'sep'")
            
    accuracy = total_correct / total_samples
    return accuracy'''

def eval_model(model, val_loader, model_type, threshold, criterion, concat_or_sep, pooling_strat):
    model.eval()
    total_correct, total_samples = 0, 0
    mse_loss = torch.nn.MSELoss()
    all_labels = []
    all_outputs = []
    
    with torch.no_grad():
        for batch_idx, (inputs, padding_mask, labels) in enumerate(val_loader):
            inputs, labels = inputs.to(device), padding_mask.to(device), labels.to(device)
            if concat_or_sep == "concat":
                outputs = model(inputs.float(), pooling_strat=pooling_strat, model_type=model_type, src_key_padding_mask=padding_mask.squeeze(1).squeeze(1))
            elif concat_or_sep == "sep":
                hla_processed, peptide_processed = inputs
                outputs = model(hla_processed.float(), peptide_processed.float(), pooling_strat=pooling_strat, model_type=model_type, src_key_padding_mask=padding_mask.squeeze(1).squeeze(1))
            else:
                raise ValueError("concat_or_sep argument must be either 'concat' or 'sep'")
            
            outputs = outputs.squeeze()
            predicted = (outputs >= threshold).long()
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            
            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())
            
            if batch_idx % 100 == 99:
                print(f'Validation Epoch: {epoch + 1}, Validation Metrics - Accuracy: {accuracy:.4f}, MSE: {mse:.4f}, AUC: {auc:.4f}, PR AUC: {pr_auc:.4f}')
                
    accuracy = total_correct / total_samples
    mse = mse_loss(torch.tensor(all_outputs), torch.tensor(all_labels).float()).item()
    auc = roc_auc_score(all_labels, all_outputs)
    pr_auc = average_precision_score(all_labels, all_outputs)
    
    return accuracy, mse, auc, pr_auc

def train_and_eval(model, train_loader, val_loader, model_type, threshold, criterion, optimizer, pooling_strat, epochs=10, concat_or_sep = "concat"):
    #print(5)
    #print(pooling_strat)
    if optimizer == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
    for epoch in range(epochs):
        # training
        train_model(model, train_loader, model_type, optimizer, criterion, epoch, concat_or_sep, pooling_strat)
        # eval
        accuracy, mse, auc, pr_auc = eval_model(model, val_loader, model_type, threshold, criterion, concat_or_sep, pooling_strat)
        #print(f'Eval Epoch: {epoch + 1}, Validation Accuracy: {accuracy:.4f}')
        #print(f'Eval Epoch: {epoch + 1}, Validation Metrics - Accuracy: {accuracy:.4f}, MSE: {mse:.4f}, AUC: {auc:.4f}, PR AUC: {pr_auc:.4f}')

    print('Finished Training and Evaluation')


In [16]:
test_model, test_alphabet = esm.pretrained.esm2_t33_650M_UR50D()
test_batch_converter = test_alphabet.get_batch_converter()

max_len = max([len(x) for x in peptide_10mer_subset.seq])
input_dim = 1280
num_layers = 6 #can change
num_heads = 8 #can change
d_ff = 2048
criterion = nn.BCELoss()
encode_choice = "esm"
fold_values = [0, 1, 2, 3, 4]

In [10]:
############################################################

In [17]:
class TransformerEncoderClassifier_Model1(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int, nlayers: int, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(ntoken, d_model) #not used in esm model!
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout, batch_first=True)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.linear = nn.Linear(d_model, 1) #bring it back to dim 1 for sigmoid classifier (if using softmax, change this to 2)

    def forward(self, src, pooling_strat = "CLS", model_type = "esm", src_mask=None, src_key_padding_mask=None) -> Tensor:
        #change src (input) shape from [batch_size, seq_len, input_dim] to [seq_len, batch_size, input_dim] (latter is more common for pytorch built-in implementations)
        #src = src.permute(1, 0, 2) -- added batch_first=True so do not need this anymore
        
        if model_type == "esm": 
            #src = self.embedding(src) #add this back in when you generalize the model for other types
            src = self.pos_encoder(src)
        '''else:
            src = self.embedding(src) #add this back in when you generalize the model for other types
            src = self.pos_encoder(src)'''
        
        #run model
        #print("model")
        #print(src.shape)
        encoded_interim = self.transformer_encoder(src, src_key_padding_mask=src_key_padding_mask)
        #print(encoded_interim.shape)
        
        ## Three ways to do this: take the representation of the [CLS] token, take the mean pooling strategy, 
        # or take the raw values themselves (unsure if the last one is actually used in practice!!)
        if pooling_strat == "mean":
            #shape of encoded_interim is [batch_size, seq_len, features]
            encoded = encoded_interim.mean(dim=1) #changed dim = 0 to dim = 1 since that is now the seq_len dim
        elif pooling_strat == "CLS":
            #assuming CLS is the first token at position 0 (assuming it is the same as SOS token for our purposes - check with Wengong [TODO])
            # With batch_first=True, the CLS token for each sequence is at position 0 along the seq_len dimension, but since we're iterating over batch, index over each item in the batch.
            #shape of encoded_interim is [batch_size, seq_len, features]
            encoded = encoded_interim[:, 0, :]
        else:
            raise ValueError(f"Invalid pooling strategy: {pooling_strat}")
        #run classifier (bring it down to one neuron then use sigmoid classifier)
        logits = self.linear(encoded)
        return torch.sigmoid(logits)

In [ ]:
print("Model 1")
for val_fold_num in fold_values:
    model = TransformerEncoderClassifier_Model1(ntoken = 22, 
                                            d_model = input_dim, 
                                            nhead = num_heads, 
                                            d_hid = d_ff, 
                                            nlayers = num_layers, 
                                            dropout = 0.1).to(device)
    initialize_param(model)
    train_loader, val_loader = get_data_loaders(peptide_10mer_subset, hla_seq_df, "label", "seq", "allele", val_fold_num, max_len, esm_model = test_model, esm_batch_converter = test_batch_converter, esm_alphabet = test_alphabet, batch_size = 32, encode_choice = encode_choice, concat_or_sep = "concat")
    train_and_eval(
        model=model, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        model_type="esm", 
        threshold=0.5, 
        criterion=criterion, 
        optimizer="Adam",
        pooling_strat = "CLS",
        epochs=10, 
        concat_or_sep="concat"
    )


Model 1
Training Epoch: 1, Batch: 100, Loss: 9.4764
Training Epoch: 1, Batch: 200, Loss: 7.2388
Training Epoch: 1, Batch: 300, Loss: 7.4418
Training Epoch: 1, Batch: 400, Loss: 7.1791
Training Epoch: 1, Batch: 600, Loss: 7.0588
Training Epoch: 1, Batch: 800, Loss: 7.0333
Training Epoch: 1, Batch: 900, Loss: 7.1468
Training Epoch: 1, Batch: 1000, Loss: 7.0358
Training Epoch: 1, Batch: 1100, Loss: 7.0375
Training Epoch: 1, Batch: 1200, Loss: 7.0005
Training Epoch: 1, Batch: 1300, Loss: 7.0351
Training Epoch: 1, Batch: 1400, Loss: 7.0085
Training Epoch: 1, Batch: 1500, Loss: 7.0794
Training Epoch: 1, Batch: 1600, Loss: 7.0090
Training Epoch: 1, Batch: 1700, Loss: 6.9969
Training Epoch: 1, Batch: 1800, Loss: 7.0651
Training Epoch: 1, Batch: 1900, Loss: 6.9875
Training Epoch: 1, Batch: 2000, Loss: 7.0027
Training Epoch: 1, Batch: 2100, Loss: 6.9776
Training Epoch: 1, Batch: 2200, Loss: 6.9832
Training Epoch: 1, Batch: 2300, Loss: 7.0155
Training Epoch: 1, Batch: 2400, Loss: 6.9611
Training 

In [ ]:
############################################################

In [15]:
class TransformerEncoderClassifier_Model2(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int, nlayers: int, dropout: float = 0.1, fine_tune_dim1: int = 512, fine_tune_dim2: int = 512):
        super().__init__()
        
        self.d_model = d_model
        self.embedding = nn.Embedding(ntoken, d_model) #not used in esm model!
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        # Adding 3 linear layers for fine-tuning ESM embeddings
        self.linear1 = nn.Linear(input_dim, fine_tune_dim1)
        self.linear2 = nn.Linear(fine_tune_dim1, fine_tune_dim2)
        self.linear3 = nn.Linear(fine_tune_dim2, input_dim)
        
        # Add dropout since you added the linear layers 
        self.dropout = nn.Dropout(dropout)
        
        # Variety of activation functions to choose from since we added 3 linear layers 
        self.relu = nn.ReLU()
        self.swish = nn.SiLU()
        self.leakyrelu = nn.LeakyReLU()
        self.prelu = nn.PReLU()
        self.elu = nn.ELU()
        self.selu = nn.SELU()
        self.gelu = nn.GELU()

        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout, batch_first=True)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.linear = nn.Linear(d_model, 1) #bring it back to dim 1 for sigmoid classifier (if using softmax, change this to 2)

    def forward(self, src, pooling_strat = "CLS", model_type = "esm", activ_fn = "ReLU", src_mask=None, src_key_padding_mask=None):
        #print(0)
        #print(pooling_strat)
        
        if activ_fn == "ReLU":
            src = self.dropout(self.relu(self.linear1(src)))
            src = self.dropout(self.relu(self.linear2(src)))
            src = self.dropout(self.relu(self.linear3(src)))
        elif activ_fn == "Swish":
            src = self.dropout(self.swish(self.linear1(src)))
            src = self.dropout(self.swish(self.linear2(src)))
            src = self.dropout(self.swish(self.linear3(src)))
        elif activ_fn == "LeakyReLU":
            src = self.dropout(self.leakyrelu(self.linear1(src)))
            src = self.dropout(self.leakyrelu(self.linear2(src)))
            src = self.dropout(self.leakyrelu(self.linear3(src)))
        elif activ_fn == "PReLU":
            src = self.dropout(self.prelu(self.linear1(src)))
            src = self.dropout(self.prelu(self.linear2(src)))
            src = self.dropout(self.prelu(self.linear3(src)))
        elif activ_fn == "ELU":
            src = self.dropout(self.elu(self.linear1(src)))
            src = self.dropout(self.elu(self.linear2(src)))
            src = self.dropout(self.elu(self.linear3(src)))
        elif activ_fn == "SELU":
            src = self.dropout(self.selu(self.linear1(src)))
            src = self.dropout(self.selu(self.linear2(src)))
            src = self.dropout(self.selu(self.linear3(src)))
        elif activ_fn == "GELU":
            src = self.dropout(self.gelu(self.linear1(src)))
            src = self.dropout(self.gelu(self.linear2(src)))
            src = self.dropout(self.gelu(self.linear3(src)))
        
        if model_type == "esm": 
            #src = self.embedding(src) #add this back in when you generalize the model for other types
            src = self.pos_encoder(src)
        else:
            #print(model_type)
            #print("incorrect")
            src = self.embedding(src) #add this back in when you generalize the model for other types
            src = self.pos_encoder(src)
            
        #run model
        encoded_interim = self.transformer_encoder(src, src_key_padding_mask=src_key_padding_mask)
        
        ## Three ways to do this: take the representation of the [CLS] token, take the mean pooling strategy, 
        # or take the raw values themselves (unsure if the last one is actually used in practice!!)
        if pooling_strat == "mean":
            encoded = encoded_interim.mean(dim=1) #changed dim = 0 to dim = 1 since that is now the seq_len dim
        elif pooling_strat == "CLS":
            #assuming CLS is the first token at position 0 (assuming it is the same as SOS token for our purposes - check with Wengong [TODO])
            # With batch_first=True, the CLS token for each sequence is at position 0 along the seq_len dimension, but since we're iterating over batch, index over each item in the batch.
            encoded = encoded_interim[:, 0, :]
        #run classifier (bring it down to one neuron then use sigmoid classifier)
        logits = self.linear(encoded)
        return torch.sigmoid(logits)

In [16]:
print("Model 2")
for val_fold_num in fold_values:
    model = TransformerEncoderClassifier_Model2(ntoken = 22, 
                                            d_model = input_dim, 
                                            nhead = num_heads, 
                                            d_hid = d_ff, 
                                            nlayers = num_layers, 
                                            dropout = 0.1,
                                            fine_tune_dim1 = 512, 
                                            fine_tune_dim2 = 512).to(device)
    initialize_param(model)
    train_loader, val_loader = get_data_loaders(peptide_10mer_subset, hla_seq_df, "label", "seq", "allele", val_fold_num, max_len, esm_model = test_model, esm_batch_converter = test_batch_converter, esm_alphabet = test_alphabet, batch_size = 32, encode_choice = encode_choice, concat_or_sep = "concat")
    train_and_eval(
        model=model, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        model_type="esm", 
        threshold=0.5, 
        criterion=criterion, 
        optimizer="Adam",
        pooling_strat = "CLS",
        epochs=5, 
        concat_or_sep="concat"
    )


Model 2
Eval Epoch: 1, Validation Accuracy: 0.6000
Eval Epoch: 2, Validation Accuracy: 0.6000
Eval Epoch: 3, Validation Accuracy: 0.6000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation
Eval Epoch: 1, Validation Accuracy: 0.6000
Eval Epoch: 2, Validation Accuracy: 0.4000
Eval Epoch: 3, Validation Accuracy: 0.4000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation
Eval Epoch: 1, Validation Accuracy: 0.4000
Eval Epoch: 2, Validation Accuracy: 0.4000
Eval Epoch: 3, Validation Accuracy: 0.4000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation
Eval Epoch: 1, Validation Accuracy: 0.6000
Eval Epoch: 2, Validation Accuracy: 0.6000
Eval Epoch: 3, Validation Accuracy: 0.6000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation


In [17]:
############################################################

In [18]:
class TransformerEncoderClassifier_Model4(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int, nlayers: int, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(ntoken, d_model) #not used in esm model!
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, batch_first=True) # batch_first=True for consistency
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout, batch_first=True)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.linear = nn.Linear(d_model, 1) #bring it back to dim 1 for sigmoid classifier (if using softmax, change this to 2)

    def forward(self, hla, peptide, pooling_strat = "CLS", model_type = "esm", src_mask=None, src_key_padding_mask=None) -> Tensor:
        #positional encoding
        if model_type == "esm": 
            #positionally encode them separately
            hla = self.pos_encoder(hla)
            peptide = self.pos_encoder(peptide)
            #print(hla.shape)
            #print(peptide.shape)
            #print(src_key_padding_mask.shape)
        else:
            hla = self.embedding(hla) #add this back in when you generalize the model for other types
            hla = self.pos_encoder(hla)
            peptide = self.embedding(peptide)
            peptide = self.pos_encoder(peptide)

        #cross-attention: hla as query, peptide as key and value
        #https://stackoverflow.com/questions/62629644/what-the-difference-between-att-mask-and-key-padding-mask-in-multiheadattnetion
        attn_output, attn_output_weights = self.multihead_attn(query=hla, key=peptide, value=peptide, key_padding_mask=src_key_padding_mask)
        #attn_output, attn_output_weights = self.multihead_attn(query=peptide, key=hla, value=hla, key_padding_mask=peptide_key_padding_mask)
        
        #print(attn_output.shape)
        
        ##encoder - CHECK/TODO: I am directly passing attn_output to the encoder but there might be other ways to do this like adding 
        #a residual layer where I add back the peptide and hla embeddings back in as well to prevent loss of details
        ##steps taken - CHECK/TODO: compute cross-attention between HLA and peptide sequences, where output represents the peptide seq in the context of the HLA seq. Feed the cross-attention output into the encoder stack. 
        #encoder layers will perform self-attention on the already combined HLA-peptide context rather than the original separate sequences --> is that okay?
        
        #TODO: ask wengong: what is the mask supposed to be here? It is supposed to cover the peptides only, but 1) that was masked in the multihead attention above and 2)the dim here are of the hla only so idk what to mask anymore in this combined representation that is the attn_output
        encoded_interim = self.transformer_encoder(attn_output, src_key_padding_mask=None)
    
        if pooling_strat == "mean":
            encoded = encoded_interim.mean(dim=1)
        elif pooling_strat == "CLS":
            #assuming CLS is the first token at position 0 (assuming it is the same as SOS token for our purposes - check with Wengong [TODO])
            encoded = encoded_interim[:, 0, :]
        #run classifier
        logits = self.linear(encoded)
        return torch.sigmoid(logits)


In [19]:
print("Model 4")
for val_fold_num in fold_values:
    model = TransformerEncoderClassifier_Model4(ntoken = 22, 
                                            d_model = input_dim, 
                                            nhead = num_heads, 
                                            d_hid = d_ff, 
                                            nlayers = num_layers, 
                                            dropout = 0.1).to(device)
    initialize_param(model)
    train_loader, val_loader = get_data_loaders(peptide_10mer_subset, hla_seq_df, "label", "seq", "allele", val_fold_num, max_len, esm_model = test_model, esm_batch_converter = test_batch_converter, esm_alphabet = test_alphabet, batch_size = 32, encode_choice = encode_choice, concat_or_sep = "sep")
    train_and_eval(
        model=model, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        model_type="esm", 
        threshold=0.5, 
        criterion=criterion, 
        optimizer="Adam",
        pooling_strat = "CLS",
        epochs=5, 
        concat_or_sep="sep"
    )


Model 4
Eval Epoch: 1, Validation Accuracy: 0.6000
Eval Epoch: 2, Validation Accuracy: 0.6000
Eval Epoch: 3, Validation Accuracy: 0.6000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation
Eval Epoch: 1, Validation Accuracy: 0.4000
Eval Epoch: 2, Validation Accuracy: 0.4000
Eval Epoch: 3, Validation Accuracy: 0.4000
Eval Epoch: 4, Validation Accuracy: 0.4000
Eval Epoch: 5, Validation Accuracy: 0.4000
Finished Training and Evaluation
Eval Epoch: 1, Validation Accuracy: 0.4000
Eval Epoch: 2, Validation Accuracy: 0.4000
Eval Epoch: 3, Validation Accuracy: 0.4000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation
Eval Epoch: 1, Validation Accuracy: 0.6000
Eval Epoch: 2, Validation Accuracy: 0.6000
Eval Epoch: 3, Validation Accuracy: 0.6000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation


In [20]:
############################################################

In [21]:
class TransformerEncoderClassifier_Model5(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int, nlayers: int, dropout: float = 0.1, fine_tune_dim1: int = 512, fine_tune_dim2: int = 512):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(ntoken, d_model) #not used in esm model!
        self.pos_encoder = PositionalEncoding(d_model, dropout)
         
        # Adding 3 linear layers for fine-tuning ESM embeddings
        self.linear1_pep = nn.Linear(input_dim, fine_tune_dim1)
        self.linear2_pep = nn.Linear(fine_tune_dim1, fine_tune_dim2)
        self.linear3_pep = nn.Linear(fine_tune_dim2, input_dim)
        
        self.linear1_hla = nn.Linear(input_dim, fine_tune_dim1)
        self.linear2_hla = nn.Linear(fine_tune_dim1, fine_tune_dim2)
        self.linear3_hla = nn.Linear(fine_tune_dim2, input_dim)
        
        # Add dropout since you added the linear layers 
        self.dropout = nn.Dropout(dropout)
        
        # Variety of activation functions to choose from since we added 3 linear layers 
        self.relu = nn.ReLU()
        self.swish = nn.SiLU()
        self.leakyrelu = nn.LeakyReLU()
        self.prelu = nn.PReLU()
        self.elu = nn.ELU()
        self.selu = nn.SELU()
        self.gelu = nn.GELU()
        
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, batch_first=True) # batch_first=True for consistency
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout, batch_first=True)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.linear = nn.Linear(d_model, 1) #bring it back to dim 1 for sigmoid classifier (if using softmax, change this to 2)

    def forward(self, hla, peptide, pooling_strat="CLS", model_type="esm", activ_fn = "ReLU", src_mask=None, src_key_padding_mask=None) -> Tensor:
        
        #run the three linear layers SEPARATELY for each hla or peptide 
        if activ_fn == "ReLU":
            activation_fn = self.relu
        elif activ_fn == "Swish":
            activation_fn = self.swish
        elif activ_fn == "LeakyReLU":
            activation_fn = self.leakyrelu
        elif activ_fn == "PReLU":
            activation_fn = self.prelu
        elif activ_fn == "ELU":
            activation_fn = self.elu
        elif activ_fn == "SELU":
            activation_fn = self.selu
        elif activ_fn == "GELU":
            activation_fn = self.gelu
            
        hla = self.dropout(activation_fn(self.linear1_hla(hla)))
        hla = self.dropout(activation_fn(self.linear2_hla(hla)))
        hla = self.dropout(activation_fn(self.linear3_hla(hla)))
        
        peptide = self.dropout(activation_fn(self.linear1_pep(peptide)))
        peptide = self.dropout(activation_fn(self.linear2_pep(peptide)))
        peptide = self.dropout(activation_fn(self.linear3_pep(peptide)))
            
        #positional encoding
        if model_type == "esm": 
            #positionally encode them separately
            hla = self.pos_encoder(hla)
            peptide = self.pos_encoder(peptide)
            #print(hla.shape)
            #print(peptide.shape)
            #print(src_key_padding_mask.shape)
        else:
            hla = self.embedding(hla) #add this back in when you generalize the model for other types
            hla = self.pos_encoder(hla)
            peptide = self.embedding(peptide)
            peptide = self.pos_encoder(peptide)

        #cross-attention: hla as query, peptide as key and value
        #https://stackoverflow.com/questions/62629644/what-the-difference-between-att-mask-and-key-padding-mask-in-multiheadattnetion
        attn_output, attn_output_weights = self.multihead_attn(query=hla, key=peptide, value=peptide, key_padding_mask=src_key_padding_mask)
        #attn_output, attn_output_weights = self.multihead_attn(query=peptide, key=hla, value=hla, key_padding_mask=peptide_key_padding_mask)

        ##encoder - CHECK/TODO: I am directly passing attn_output to the encoder but there might be other ways to do this like adding 
        #a residual layer where I add back the peptide and hla embeddings back in as well to prevent loss of details
        ##steps taken - CHECK/TODO: compute cross-attention between HLA and peptide sequences, where output represents the peptide seq in the context of the HLA seq. Feed the cross-attention output into the encoder stack. 
        #encoder layers will perform self-attention on the already combined HLA-peptide context rather than the original separate sequences --> is that okay?
        #TODO: ask wengong: what is the mask supposed to be here? It is supposed to cover the peptides only, but 1) that was masked in the multihead attention above and 2)the dim here are of the hla only so idk what to mask anymore in this combined representation that is the attn_output
        encoded_interim = self.transformer_encoder(attn_output, src_key_padding_mask=None)
    
        if pooling_strat == "mean":
            encoded = encoded_interim.mean(dim=1)
        elif pooling_strat == "CLS":
            #assuming CLS is the first token at position 0 (assuming it is the same as SOS token for our purposes - check with Wengong [TODO])
            encoded = encoded_interim[:, 0, :]
        #run classifier
        logits = self.linear(encoded)
        return torch.sigmoid(logits)

In [22]:
print("Model 5")
for val_fold_num in fold_values:
    model = TransformerEncoderClassifier_Model5(ntoken = 22, 
                                            d_model = input_dim, 
                                            nhead = num_heads, 
                                            d_hid = d_ff, 
                                            nlayers = num_layers, 
                                            dropout = 0.1,
                                            fine_tune_dim1 = 512, 
                                            fine_tune_dim2 = 512).to(device)
    initialize_param(model)
    train_loader, val_loader = get_data_loaders(peptide_10mer_subset, hla_seq_df, "label", "seq", "allele", val_fold_num, max_len, esm_model = test_model, esm_batch_converter = test_batch_converter, esm_alphabet = test_alphabet, batch_size = 32, encode_choice = encode_choice, concat_or_sep = "sep")
    train_and_eval(
        model=model, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        model_type="esm", 
        threshold=0.5, 
        criterion=criterion, 
        optimizer="Adam",
        pooling_strat = "CLS",
        epochs=5, 
        concat_or_sep="sep"
    )


Model 5
Eval Epoch: 1, Validation Accuracy: 0.6000
Eval Epoch: 2, Validation Accuracy: 0.6000
Eval Epoch: 3, Validation Accuracy: 0.6000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation
Eval Epoch: 1, Validation Accuracy: 0.6000
Eval Epoch: 2, Validation Accuracy: 0.4000
Eval Epoch: 3, Validation Accuracy: 0.4000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation
Eval Epoch: 1, Validation Accuracy: 0.4000
Eval Epoch: 2, Validation Accuracy: 0.4000
Eval Epoch: 3, Validation Accuracy: 0.4000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.4000
Finished Training and Evaluation
Eval Epoch: 1, Validation Accuracy: 0.6000
Eval Epoch: 2, Validation Accuracy: 0.4000
Eval Epoch: 3, Validation Accuracy: 0.6000
Eval Epoch: 4, Validation Accuracy: 0.6000
Eval Epoch: 5, Validation Accuracy: 0.6000
Finished Training and Evaluation
